In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import calendar
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'bi-presto.serving.data.production.internal'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
pickup_location_hex_8, pickup_location_latitude, pickup_location_longitude, drop_location_hex_8, drop_location_latitude, drop_location_longitude,
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20230320'
and yyyymmdd <= '20230402'
and city_name in ('Bangalore')
and service_obj_service_name = 'Link'
and hour in ('08', '09', '10', '11', '17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [ ]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason
0,640b3d012af2720155ca3b06,NaN,8860169607fffff,13.101949,77.572144,Bangalore,8860169607fffff,13.101949,8860169607fffff,13.101965,77.572178,8860145985fffff,13.052620,77.488878,77.572144,195153,19,1945,Link,19:59,5,20230310,customerCancelled,order cancelled before rider accepted
1,640b3d1cd980fd79fc8a137a,4.663,8861892097fffff,12.942293,77.695770,Bangalore,8861892097fffff,12.940962,8861892097fffff,12.941810,77.693130,8861892097fffff,12.942293,77.695770,77.693673,195220,19,1945,Link,19:59,5,20230310,dropped,None
2,640b3d232af2720155ca3b37,1.617,88618925d7fffff,12.929673,77.591629,Bangalore,8861892eb9fffff,13.001411,8860145b69fffff,12.991840,77.601647,88618925d7fffff,12.929673,77.591629,77.617979,195227,19,1945,Link,19:59,5,20230310,dropped,None
3,640b3de0d980fd79fc8a1498,NaN,8861892567fffff,12.954659,77.679517,Bangalore,8861892097fffff,12.939914,8861892097fffff,12.939911,77.694793,8861892085fffff,12.949613,77.722481,77.694809,195536,19,1945,Link,19:59,5,20230310,expired,None
4,640b3ec606d5be4a71ed1ec2,0.302,8861892429fffff,12.906255,77.638718,Bangalore,8861892513fffff,12.929811,8861892513fffff,12.929812,77.633322,8861892429fffff,12.906255,77.638718,77.633324,195926,19,1945,Link,19:59,5,20230310,dropped,None


In [39]:
bangalore_tam = pd.read_csv('bangalore.csv')
bangalore_tam_hexes = bangalore_tam['pickup_location_hex_8'].unique().tolist()

FileNotFoundError: [Errno 2] No such file or directory: 'bangalore.csv'

In [ ]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count
0,640b3d012af2720155ca3b06,NaN,8860169607fffff,13.101949,77.572144,Bangalore,8860169607fffff,13.101949,8860169607fffff,13.101965,77.572178,8860145985fffff,13.052620,77.488878,77.572144,195153,19,1945,Link,19:59,5,20230310,customerCancelled,order cancelled before rider accepted,53,51,10,1
1,640b3d1cd980fd79fc8a137a,4.663,8861892097fffff,12.942293,77.695770,Bangalore,8861892097fffff,12.940962,8861892097fffff,12.941810,77.693130,8861892097fffff,12.942293,77.695770,77.693673,195220,19,1945,Link,19:59,5,20230310,dropped,None,20,52,10,1
2,640b3d232af2720155ca3b37,1.617,88618925d7fffff,12.929673,77.591629,Bangalore,8861892eb9fffff,13.001411,8860145b69fffff,12.991840,77.601647,88618925d7fffff,12.929673,77.591629,77.617979,195227,19,1945,Link,19:59,5,20230310,dropped,None,27,52,10,1
3,640b3de0d980fd79fc8a1498,NaN,8861892567fffff,12.954659,77.679517,Bangalore,8861892097fffff,12.939914,8861892097fffff,12.939911,77.694793,8861892085fffff,12.949613,77.722481,77.694809,195536,19,1945,Link,19:59,5,20230310,expired,None,36,55,10,1
4,640b3ec606d5be4a71ed1ec2,0.302,8861892429fffff,12.906255,77.638718,Bangalore,8861892513fffff,12.929811,8861892513fffff,12.929812,77.633322,8861892429fffff,12.906255,77.638718,77.633324,195926,19,1945,Link,19:59,5,20230310,dropped,None,26,59,10,1


In [23]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Bangalore')
and resolution = 8
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [24]:
cluster_mapping = df_cluster_copy.copy()
cluster_mapping = cluster_mapping.rename(columns = {'hex_id':'pickup_location_hex_8'})
df = df.merge(cluster_mapping[['pickup_location_hex_8', 'cluster']], on = 'pickup_location_hex_8', how = 'left')
cluster_mapping = cluster_mapping.rename(columns = {'pickup_location_hex_8':'drop_location_hex_8'})
df = df.rename(columns = {'cluster':'pickup_cluster'})
df = df.merge(cluster_mapping[['drop_location_hex_8', 'cluster']], on = 'drop_location_hex_8', how = 'left')
df = df.rename(columns = {'cluster':'drop_cluster'})
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster
0,640b3d012af2720155ca3b06,NaN,8860169607fffff,13.101949,77.572144,Bangalore,8860169607fffff,13.101949,8860169607fffff,13.101965,77.572178,8860145985fffff,13.052620,77.488878,77.572144,195153,19,1945,Link,19:59,5,20230310,customerCancelled,order cancelled before rider accepted,53,51,10,1,Yelahanka New Town,Dasarahalli
1,640b3d1cd980fd79fc8a137a,4.663,8861892097fffff,12.942293,77.695770,Bangalore,8861892097fffff,12.940962,8861892097fffff,12.941810,77.693130,8861892097fffff,12.942293,77.695770,77.693673,195220,19,1945,Link,19:59,5,20230310,dropped,None,20,52,10,1,Kadubeesanahalli ORR,Kadubeesanahalli ORR
2,640b3d232af2720155ca3b37,1.617,88618925d7fffff,12.929673,77.591629,Bangalore,8861892eb9fffff,13.001411,8860145b69fffff,12.991840,77.601647,88618925d7fffff,12.929673,77.591629,77.617979,195227,19,1945,Link,19:59,5,20230310,dropped,None,27,52,10,1,Train - Cantonment,Jayanagar 4th Block
3,640b3de0d980fd79fc8a1498,NaN,8861892567fffff,12.954659,77.679517,Bangalore,8861892097fffff,12.939914,8861892097fffff,12.939911,77.694793,8861892085fffff,12.949613,77.722481,77.694809,195536,19,1945,Link,19:59,5,20230310,expired,None,36,55,10,1,Kadubeesanahalli ORR,Columbia Asia Whitefield
4,640b3ec606d5be4a71ed1ec2,0.302,8861892429fffff,12.906255,77.638718,Bangalore,8861892513fffff,12.929811,8861892513fffff,12.929812,77.633322,8861892429fffff,12.906255,77.638718,77.633324,195926,19,1945,Link,19:59,5,20230310,dropped,None,26,59,10,1,Koramangala - ST Bed,ITI Layout HSR


In [25]:
#Filter the df on morning and evening peak hours
df_morning = df[df['hour'].isin(['08', '09', '10', '11'])]
df_evening = df[df['hour'].isin(['17', '18', '19', '20', '21'])]

#Groupby on hex and quarter hour to get the number of orders coming in every hex
df_morning_grouped = df_morning.groupby(['yyyymmdd', 'hour', 'pickup_location_hex_8'])['order_id'].nunique().reset_index()
df_evening_grouped = df_evening.groupby(['yyyymmdd', 'hour', 'pickup_location_hex_8'])['order_id'].nunique().reset_index()
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped.head()

,yyyymmdd,hour,pickup_location_hex_8,num_of_orders
0,20230306,17,8860144b4dfffff,1
1,20230306,17,8860144b51fffff,4
2,20230306,17,8860144b59fffff,1
3,20230306,17,8860144b5bfffff,1
4,20230306,17,8860145121fffff,9


In [26]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'hour', 'pickup_location_hex_8'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'hour', 'pickup_location_hex_8'], how = 'left')

In [27]:
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])

In [11]:
#Query data for mismatch

q_online_captains = """
select * from pricing.cluster_temporal_hotness
where service_detail_id = '57370b61a6855d70057417d1'
"""


#Load data into pandas table
df_mismatch_copy = pd.read_sql(q_online_captains, connection)

In [28]:
#View the dataset
df_mismatch = df_mismatch_copy.copy()
df_mismatch.head()

,hotness_category,pickupclusters,week_period,time_period,temporal_id,total_footfall,total_revenue,hex_count,total_revenue_normalized,normalized_revenue_contribution_percentage,cum_revenue_contribution_percentage,revenue_contribution_percentile_rank,temporal_stress,temporal_mismatch,spatial_stress,spatial_mismatch,cluster_hotness_score,run_date,strategy_version,service_detail_id
0,medium,AECS Layout Brookefield,Wednesday,2300,Wednesday-2300,0,0.0,0,0.0,0.0,0.0,0.0,0.673706,0.618777,0.187500,0.156250,0.409058,20230314,HOT_COLD_3_0,57370b61a6855d70057417d1
1,medium,AECS Layout Brookefield,Wednesday,0400,Wednesday-0400,0,0.0,0,0.0,0.0,0.0,0.0,0.601715,0.597346,0.178571,0.142857,0.380122,20230314,HOT_COLD_3_0,57370b61a6855d70057417d1
2,cold,AECS Layout Brookefield,Wednesday,0000,Wednesday-0000,0,0.0,0,0.0,0.0,0.0,0.0,0.400185,0.405103,0.093750,0.083929,0.245742,20230314,HOT_COLD_3_0,57370b61a6855d70057417d1
3,medium,AECS Layout Brookefield,Wednesday,0600,Wednesday-0600,0,0.0,0,0.0,0.0,0.0,0.0,0.708769,0.693079,0.327679,0.265179,0.498676,20230314,HOT_COLD_3_0,57370b61a6855d70057417d1
4,medium,AECS Layout Brookefield,Wednesday,1700,Wednesday-1700,0,0.0,0,0.0,0.0,0.0,0.0,0.586093,0.630946,0.236607,0.278571,0.433054,20230314,HOT_COLD_3_0,57370b61a6855d70057417d1


In [30]:
df_mismatch = df_mismatch[df_mismatch['run_date'] == df_mismatch['run_date'].max()]

In [31]:
#rename column
df_mismatch = df_mismatch.rename(columns = {'week_period':'weekday_name', 'pickupclusters':'pickup_cluster'})
df_mismatch['hour'] = df_mismatch['time_period'].apply(lambda x: x[:2])
df_mismatch.head()

,hotness_category,pickup_cluster,weekday_name,time_period,temporal_id,total_footfall,total_revenue,hex_count,total_revenue_normalized,normalized_revenue_contribution_percentage,cum_revenue_contribution_percentage,revenue_contribution_percentile_rank,temporal_stress,temporal_mismatch,spatial_stress,spatial_mismatch,cluster_hotness_score,run_date,strategy_version,service_detail_id,hour
0,medium,AECS Layout Brookefield,Wednesday,2300,Wednesday-2300,0,0.0,0,0.0,0.0,0.0,0.0,0.673706,0.618777,0.187500,0.156250,0.409058,20230314,HOT_COLD_3_0,57370b61a6855d70057417d1,23
1,medium,AECS Layout Brookefield,Wednesday,0400,Wednesday-0400,0,0.0,0,0.0,0.0,0.0,0.0,0.601715,0.597346,0.178571,0.142857,0.380122,20230314,HOT_COLD_3_0,57370b61a6855d70057417d1,04
2,cold,AECS Layout Brookefield,Wednesday,0000,Wednesday-0000,0,0.0,0,0.0,0.0,0.0,0.0,0.400185,0.405103,0.093750,0.083929,0.245742,20230314,HOT_COLD_3_0,57370b61a6855d70057417d1,00
3,medium,AECS Layout Brookefield,Wednesday,0600,Wednesday-0600,0,0.0,0,0.0,0.0,0.0,0.0,0.708769,0.693079,0.327679,0.265179,0.498676,20230314,HOT_COLD_3_0,57370b61a6855d70057417d1,06
4,medium,AECS Layout Brookefield,Wednesday,1700,Wednesday-1700,0,0.0,0,0.0,0.0,0.0,0.0,0.586093,0.630946,0.236607,0.278571,0.433054,20230314,HOT_COLD_3_0,57370b61a6855d70057417d1,17


In [33]:
df_morning = df_morning.merge(df_mismatch[['weekday_name', 'hour', 'pickup_cluster', 'cluster_hotness_score']],
                               on = ['weekday_name', 'hour', 'pickup_cluster'], how = 'left')
df_evening = df_evening.merge(df_mismatch[['weekday_name', 'hour', 'pickup_cluster', 'cluster_hotness_score']],
                               on = ['weekday_name', 'hour', 'pickup_cluster'], how = 'left')
df_evening.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster,num_of_orders,weekday_name,cluster_hotness_score
0,640b3d012af2720155ca3b06,NaN,8860169607fffff,13.101949,77.572144,Bangalore,8860169607fffff,13.101949,8860169607fffff,13.101965,77.572178,8860145985fffff,13.052620,77.488878,77.572144,195153,19,1945,Link,19:59,5,20230310,customerCancelled,order cancelled before rider accepted,53,51,10,1,Yelahanka New Town,Dasarahalli,4,Friday,0.173076
1,640b3d1cd980fd79fc8a137a,4.663,8861892097fffff,12.942293,77.695770,Bangalore,8861892097fffff,12.940962,8861892097fffff,12.941810,77.693130,8861892097fffff,12.942293,77.695770,77.693673,195220,19,1945,Link,19:59,5,20230310,dropped,None,20,52,10,1,Kadubeesanahalli ORR,Kadubeesanahalli ORR,106,Friday,0.868586
2,640b3d232af2720155ca3b37,1.617,88618925d7fffff,12.929673,77.591629,Bangalore,8861892eb9fffff,13.001411,8860145b69fffff,12.991840,77.601647,88618925d7fffff,12.929673,77.591629,77.617979,195227,19,1945,Link,19:59,5,20230310,dropped,None,27,52,10,1,Train - Cantonment,Jayanagar 4th Block,21,Friday,0.665234
3,640b3de0d980fd79fc8a1498,NaN,8861892567fffff,12.954659,77.679517,Bangalore,8861892097fffff,12.939914,8861892097fffff,12.939911,77.694793,8861892085fffff,12.949613,77.722481,77.694809,195536,19,1945,Link,19:59,5,20230310,expired,None,36,55,10,1,Kadubeesanahalli ORR,Columbia Asia Whitefield,106,Friday,0.868586
4,640b3ec606d5be4a71ed1ec2,0.302,8861892429fffff,12.906255,77.638718,Bangalore,8861892513fffff,12.929811,8861892513fffff,12.929812,77.633322,8861892429fffff,12.906255,77.638718,77.633324,195926,19,1945,Link,19:59,5,20230310,dropped,None,26,59,10,1,Koramangala - ST Bed,ITI Layout HSR,83,Friday,0.771735


In [36]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'weekday_name', 'cluster_hotness_score', 'order_status']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'weekday_name', 'cluster_hotness_score', 'order_status']]

In [37]:
df_evening_filter.to_csv('bangalore_demand_supply_evening_vw2.csv', index = False)
df_morning_filter.to_csv('bangalore_demand_supply_morning_vw2.csv', index = False)

In [18]:
#Make a copy of the dataset and view it
df_online_captains = df_online_captains_copy.copy()
df_online_captains.head()

,yyyymmdd,hour,cluster,Online_Captains
0,20230301,17,Nungambakkam,237
1,20230228,17,Nungambakkam,231
2,20230301,21,Teynampet,175
3,20230228,20,T Nagar,322
4,20230301,18,Kotturpuram,66


In [19]:
df_online_captains.groupby(['yyyymmdd', 'hour'])['Online_Captains'].sum().reset_index()

,yyyymmdd,hour,Online_Captains
0,20230227,17,1338
1,20230227,18,1595
2,20230227,19,1795
3,20230227,20,1689
4,20230227,21,1399
5,20230228,17,1257
6,20230228,18,1519
7,20230228,19,1787
8,20230228,20,1624
9,20230228,21,1371


In [20]:
#Rename cluster column
df_online_captains = df_online_captains.rename(columns = {'cluster':'pickup_cluster'})
df_online_captains.head()

,yyyymmdd,hour,pickup_cluster,Online_Captains
0,20230301,17,Nungambakkam,237
1,20230228,17,Nungambakkam,231
2,20230301,21,Teynampet,175
3,20230228,20,T Nagar,322
4,20230301,18,Kotturpuram,66


In [21]:
#Merge online captains with demand data
df_morning_grouped = df_morning_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_evening_grouped = df_evening_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_morning_grouped['demand_to_supply_ratio'] = round(df_morning_grouped['num_of_orders']/df_morning_grouped['Online_Captains'], 4)
df_evening_grouped['demand_to_supply_ratio'] = round(df_evening_grouped['num_of_orders']/df_evening_grouped['Online_Captains'], 4)

df_morning_grouped = df_morning_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped = df_evening_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped.head()

,yyyymmdd,hour,pickup_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio
169,20230302,18,Thousand Lights Metro,92,43,2.1395
119,20230301,18,Thousand Lights Metro,87,50,1.7400
164,20230302,18,Nungambakkam,392,235,1.6681
100,20230301,17,Alwarpet,71,44,1.6136
110,20230301,18,Alwarpet,118,76,1.5526


In [27]:
df_evening_grouped.groupby(['hour'])['demand_to_supply_ratio'].mean()

hour
17    0.918695
18    1.022725
19    0.714440
20    0.515355
21    0.482165
Name: demand_to_supply_ratio, dtype: float64

In [22]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,quarter_hour,service_obj_service_name,time_bucket,weekday,order_status,cancel_reason,second,minute,date,count,drop_cluster,num_of_orders,yyyymmdd,hour,pickup_cluster,Online_Captains,demand_to_supply_ratio


In [23]:
#Create a column for weekdays
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,quarter_hour,service_obj_service_name,time_bucket,weekday,order_status,cancel_reason,second,minute,date,count,drop_cluster,num_of_orders,yyyymmdd,hour,pickup_cluster,Online_Captains,demand_to_supply_ratio,weekday_name


In [24]:
num_of_hex_morning = df_morning.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_evening = df_evening.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_morning = num_of_hex_morning.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})
num_of_hex_evening = num_of_hex_evening.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})

df_morning = df_morning.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_evening = df_evening.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_morning['online_captains_per_hex'] = round(df_morning['Online_Captains']/df_morning['num_of_hexes'], 4)
df_evening['online_captains_per_hex'] = round(df_evening['Online_Captains']/df_evening['num_of_hexes'], 4)

In [25]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]

In [26]:
df_evening_filter.to_csv('chennai_demand_supply_evening.csv', index = False)